In [2]:
import json
import sys
sys.path.append('/home/nbuser/library/')
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import seaborn as sns

%matplotlib inline

Importing dataset as dataframe

In [3]:
df = pd.read_csv('Dataset.csv')

In [4]:
x = df.head() #for future testing

API Call: Attaching 2018 Census Data

In [5]:
def get_data(lon, lat):
    """Use longitudea and latitude to query the API for data"""
    url = 'https://koordinates.com/services/query/v1/vector.json'
    params = {'key' : 'bbe72a48c4814491829c9d02c005ac2b', 'layer' : 104612, 'x': lon, 'y': lat}
    response = requests.get(url, params=params)
    if response.status_code != 200:
        return response.status_code
    return response.json()['vectorQuery']['layers']['104612']['features'][0]['properties']['C18_CURPop']

In [6]:
get_data(174.922912, -37.063672) #example

129

In [7]:
x['2018 population'] = x.apply(lambda row: get_data(row['Longitude'], row['Latitude']), axis = 1)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
x

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,2018 population
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa,174
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka,129
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka,129
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga,120
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park,231


In [ ]:
df['2018 population'] = df.apply(lambda row: get_data(row['Longitude'], row['Latitude']), axis = 1)

Adding Deprivation Index

In [ ]:
!curl 'https://www.otago.ac.nz/wellington/otago730418.txt' -o data.csv

In [ ]:
df2 = pd.read_csv('data.csv', delimiter='\t')

In [ ]:
df2.head()

In [ ]:
df3 = df.merge(df2, how='left', left_on='SA1', right_on='SA12018_code').drop(['SA22018_code', 'SA22018_name', 'URPopnSA1_2018'], axis=1)

In [ ]:
df3.head()

Cleaning the data

In [ ]:
df3.info()

In [ ]:
#Land area is integer, not object
def con(x):
    a = ""
    for i in x:
        if i == " ":
            break
        a += i
    return float(a)

In [ ]:
df3['Land area'] = df3['Land area'].apply(con)

In [ ]:
df3.info()

In [ ]:
df3.describe()

In [ ]:
#2 Bathroom values are missing, these will be filled by building a classifier using "Bedrooms", "Land area" and "CV"
logreg = LogisticRegression()
df3X = df3[['Bedrooms', 'Land area', 'CV']]
df3X_train = df3.dropna()[['Bedrooms', 'Land area', 'CV']]
df3X_test = df3[df3['Bathrooms'].isnull()][['Bedrooms', 'Land area', 'CV']]
df3y_train = df3.dropna()['Bathrooms']
logreg.fit(df3X_train, df3y_train)
pred = logreg.predict(df3X_test)

In [ ]:
df3.loc[df3['Bathrooms'].isnull(), 'Bathrooms'] = pred

In [ ]:
df3.describe()

In [ ]:
#Number of bathrooms should be an integer
df3['Bathrooms'] = df3['Bathrooms'].astype(int)

In [ ]:
df3.describe()
df3.to_csv('Dataset.csv')

Now we can perform some initial data exploration

In [ ]:
df3_temp = df3.drop(['Latitude', 'Longitude', 'SA1', 'SA12018_code', 'NZDep2018', 'Address'], axis=1)

In [ ]:
sns.pairplot(df3_temp, hue='Suburbs')

KeyboardInterrupt: 